In [ ]:
# !pip install stable_baselines3
# !pip install langgraph
# !pip install langchain_huggingface
# !pip install transformers

In [ ]:

import transformers,torch
import langgraph
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback

from langchain.tools import tool
from stable_baselines3 import PPO
import numpy as np
import base_env
import data_utils
import pandas as pd
import numpy as np


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

In [ ]:
# -----------------------------
# 0️⃣ Prepare Data
# -----------------------------
data_dict = data_utils.prepare_data_dict(
    sku_file="skus.csv",
    truck_file="trucks.csv",
    route_file="routes.csv",
    external_file="external_factors.csv"
)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# data_dict

In [ ]:
# -----------------------------
# 1️⃣ Custom Step Logger Callback
# -----------------------------
class StepLoggerCallback(BaseCallback):
    def __init__(self, check_freq=1000, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            print(f"➡️ Training step: {self.n_calls}")
        return True


In [ ]:
# -----------------------------
# 2️⃣ Create Vectorized Training Environment
# -----------------------------
num_train_envs = 8
def make_train_env():
    def _init():
        return base_env.TruckRoutingEnv(data_dict)
    return _init


In [ ]:
# Windows-safe: use DummyVecEnv
train_env = DummyVecEnv([make_train_env() for _ in range(num_train_envs)])


In [ ]:
# -----------------------------
# 3️⃣ Train PPO Model
# -----------------------------
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    n_steps=64,
    batch_size=32,
)

Using cpu device


In [ ]:
callback = StepLoggerCallback(check_freq=2000)
print("🚀 Starting training...")
model.learn(total_timesteps=1, callback=callback)  # reduced timesteps
print("✅ Training complete")

# -----------------------------
# 4️⃣ Parallel Evaluation (max 5000 steps)
# -----------------------------
print("🎯 Evaluating model and saving results...")

num_eval_envs = 1   # 1 env for faster evaluation
max_eval_steps = 5000


🚀 Starting training...
----------------------------
| time/              |     |
|    fps             | 70  |
|    iterations      | 1   |
|    time_elapsed    | 7   |
|    total_timesteps | 512 |
----------------------------
✅ Training complete
🎯 Evaluating model and saving results...


In [ ]:

@tool("ppo_truck_decision")
def choose_truck_route(state: dict) -> dict:
    """
    Decide optimal truck-route combo given current logistics state.
    Input: {'features': [float, float, ...]}
    Output: {'action': int, 'action_desc': str}
    """
    obs = np.array(state["features"]).reshape(1, -1)
    action, _ = model.predict(obs, deterministic=True)

    return {
        "action": int(action),
        "action_desc": f"Chosen action ID {int(action)} based on PPO policy"
    }

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEndpoint
import os

llm = HuggingFaceEndpoint(
    repo_id="tiiuae/falcon-rw-1b",       # replace with the Falcon RW 1B repo ID or whichever model you want
    task="text-generation",             # specify task type if needed
    max_new_tokens=128,
    temperature=0.0,
    top_k=50,                           # optional: sample settings
    top_p=0.9,                          # optional
    huggingfacehub_api_token="hf_
)

def explain_decision(input_data: dict):
    action = input_data["action"]
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert logistics analyst."),
        ("human", f"The RL agent selected action {action}. Explain what this might represent and why it makes sense.")
    ])
    response = llm(prompt.format_messages())
    return {"explanation": response.content}

In [ ]:
# Step 4: Build the LangGraph Pipeline
from langgraph.graph import StateGraph

# from ppo_node import choose_truck_route
# from llm_node import explain_decision

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver


In [ ]:
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
class TruckRoutingEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 4} # what is meta data ?

    def __init__(self, data_dict):
        super().__init__()

        # Load data from prepared dict
        self.skus = data_dict["skus"]
        self.trucks = data_dict["trucks"]
        self.routes = data_dict["routes"]
        self.external = data_dict["external"]

        # Define action space: all truck x route combinations
        self.num_trucks = len(self.trucks)
        self.num_routes = len(self.routes)
        self.action_space = spaces.Discrete(self.num_trucks * self.num_routes)

        # Observation space: 10-d random vector (can extend with real features)
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(10,), dtype=np.float32) # what is observation space

        # Internal state
        self.current_step = 0
        self.max_steps = len(self.skus)
        self.total_reward = 0

        # Keep track of current SKU for evaluation logging
        self.current_sku_idx = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0  ## what is current step?
        self.total_reward = 0
        self.current_sku_idx = 0
        return self._get_obs(), {}

    def step(self, action):
        # Check for out-of-bounds step
        if self.current_step >= self.max_steps:
            obs = self._get_obs()
            return obs, 0.0, True,False, {}

        # Current SKU
        sku = self.skus.iloc[self.current_step]

        # Decode action into truck and route indices
        truck_idx = action // self.num_routes
        route_idx = action % self.num_routes
        truck = self.trucks.iloc[truck_idx]
        route = self.routes.iloc[route_idx]

        # Reward calculation
        reward = 0.0

        # 1️⃣ Capacity check (weight)
        if sku["weight_kg"] <= truck["capacity_kg"]:
            reward += 5
        else:
            reward -= 10

        # 2️⃣ Travel time penalty (simplified using expected travel time)
        travel_time_hours = route["expected_travel_secs"] / 3600
        reward -= travel_time_hours

        # 3️⃣ Fuel cost penalty
        fuel_per_km = truck["fuel_capacity_l"] / 100  # assume arbitrary conversion
        fuel_cost = route["distance_km"] * fuel_per_km
        reward -= fuel_cost

        # 4️⃣ Weather penalty if external factors exist
        if not self.external.empty:
            weather_idx = self.current_step % len(self.external)
            snow = self.external.iloc[weather_idx].get("snow_cm", 0)
            rain = self.external.iloc[weather_idx].get("rain_mm", 0)
            if snow > 0:
                reward -= 5
            if rain > 10:
                reward -= 2

        # Update environment state
        self.total_reward += reward
        self.current_step += 1
        self.current_sku_idx = self.current_step - 1
        done = self.current_step >= self.max_steps

        # Observation
        obs = self._get_obs()

        # Info for logging
        info = {
            "sku_id": sku.get("sku_id", self.current_sku_idx),
            "truck_id": truck.get("truck_id", truck_idx),
            "route_id": route.get("route_id", route_idx)
        }

        return obs, reward,done,True,info

    def _get_obs(self):
        # 10-d random vector for now (replace with meaningful features if needed)
        return np.random.rand(10).astype(np.float32)

    def render(self):
        print(f"Step {self.current_step}, Total Reward: {self.total_reward}")


In [ ]:
num_eval_envs = 1
def make_eval_env(idx):
    def _init():
        return TruckRoutingEnv(data_dict)
    return _init

eval_env2 = DummyVecEnv([make_eval_env(i) for i in range(num_eval_envs)])



In [ ]:
# node -1 (ppo node)
def rl_decision_node(state):
    obs = state["obs"]
    env=state["env"]
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done,infos = env.step(action)

    state["obs"] = obs # new obs created & env variables updated
    state["reward"]=reward[0] # new reward created


    num_routes = len(data_dict["routes"])
    truck_idx = action // num_routes
    route_idx = action % num_routes

    # Map indices to actual IDs
    truck_name = (
        data_dict["trucks"].iloc[truck_idx]["truck_id"]
        if truck_idx < len(data_dict["trucks"])
        else f"Truck_{truck_idx}"
    )
    route_name = (
        data_dict["routes"].iloc[route_idx]["route_id"]
        if route_idx < len(data_dict["routes"])
        else f"Route_{route_idx}"
    )


    state["action"] = {"truck": truck_name.values[0], "route": route_name.values[0]}
    # state_action={"truck": truck_name, "route": route_name}
    print(f"🤖 RL chose → Truck: {truck_name.values[0]}, Route: {route_name.values[0]}")


    # Update moving average
    state["avg_reward"] = 0.8 * state.get("avg_reward") + 0.2 * state["reward"]

    print(f"📊 Feedback collected — reward: {state["reward"]:.2f}, avg: {state['avg_reward']:.2f}")

    return state



# node -2
def check_perf(state):
    threshold =  -50 # example: retrain if avg_reward too low
    if state["avg_reward"] < threshold:
        print("⚠️ Low performance detected. Initiating retraining...")
        state["retrain_flag"] = True
    else:
        print("✅ Performance acceptable, no retraining needed.")
        state["retrain_flag"] = False
    return state


# node -3
def retrain_model(state):
    if state["retrain_flag"]:
        print("🔁 Retraining PPO model on new simulated data...")
        # Example retraining step — replace with actual env training
        env = DummyVecEnv([lambda: TruckRoutingEnv(state["data_dict"])])
        model = PPO("MlpPolicy", env, verbose=0)
        model.learn(total_timesteps=5000)
        model.save("ppo_truck_routing")
        print("✅ Retraining complete, model updated.")
    else:
        print("⏩ Skipping retrain, model is fine.")
    return state


In [ ]:
from google.colab import userdata
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528",      # replace with the Falcon RW 1B repo ID or whichever model you want
    task="text-generation",             # specify task type if needed
    max_new_tokens=128,
    temperature=0.0,
    top_k=50,                           # optional: sample settings
    top_p=0.9,                          # optional
    huggingfacehub_api_token=userdata.get('N1')
)
# node -4
# def explain_decision(state):
#     action = state["action"]
#     query = f"The RL model chose Truck {action['truck']} for Route {action['route']}. Explain the likely reasoning."
#     explanation = llm.invoke(query).content
#     state["decision_explained"] = explanation
#     print(f"🧠 LLM Explanation: {explanation}")
#     return state

In [ ]:

def explain_decision(state):
    action = state["action"]
    query = f"The RL model chose Truck {action['truck']} for Route {action['route']}. Explain the likely reasoning."
    model_name = "tiiuae/falcon-rw-1b"
    tokenizer = AutoTokenizer.from_pretrained(model_name,padding=True, truncation=True, return_tensors="pt")

    llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto")

    inputs = tokenizer(query, return_tensors="pt").to(llm.device)
    attention_mask = inputs.attention_mask
    outputs = llm.generate(
        inputs["input_ids"],
        max_new_tokens=50,
        do_sample=True,
        top_k=50,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id, attention_mask=attention_mask
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

    explanation=tokenizer.decode(outputs[0], skip_special_tokens=True)
    state["decision_explained"] = explanation
    # print(f"🧠 LLM Explanation: {explanation}")
    return state

In [ ]:
def dispatch_action(state):
    action = state["action"]

    state["total_rewards"] += state["reward"]
    result_summary = {
        "status": "dispatched",
        "truck": action["truck"],
        "route": action["route"],
        "timestamp": pd.Timestamp.now()
    }
    # print(result_summary)
    state["result"] = result_summary
    # print(f"🚚 Dispatched Truck {action['truck']} on Route {action['route']}")
    return state

In [ ]:

class AgentState(TypedDict):
    data_dict: dict
    model: PPO
    env: TruckRoutingEnv
    obs: AnyMessage
    action: dict
    decision_explained: str
    result: AnyMessage
    reward: float
    avg_reward: float
    total_rewards: float
    retrain_flag: bool

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph import StateGraph, START, END
graph = StateGraph(AgentState)

In [ ]:
# Create the graph
def build_continuous_graph():
    graph = StateGraph(AgentState)
    graph.add_node("choose_action", rl_decision_node)
    graph.add_node("evaluate_perf", check_perf)
    graph.add_node("retrain_model", retrain_model)
    graph.add_node("explain_decision", explain_decision)
    graph.add_node("dispatch", dispatch_action)

    # Flow of operations
    graph.add_edge(START, "choose_action")
    graph.add_edge("choose_action", "evaluate_perf")
    graph.add_conditional_edges(
        "evaluate_perf",
        lambda s: "retrain_model" if s["retrain_flag"] else "explain_decision",
        ["retrain_model", "explain_decision"]
    )
    graph.add_edge("explain_decision", "dispatch")
    graph.add_edge("dispatch", END)

    graph.set_entry_point("choose_action")
    graph.set_finish_point("dispatch")
    return graph

In [ ]:
state = {
    "data_dict": data_dict,
    "model": model,
    "env": eval_env2,
    "obs": eval_env2.reset(),
    # "observation": None,
    "action": None,
    "decision_explained": "",
    "result": None,
    "reward": 0.0,                # latest reward or feedback score
    "avg_reward": 0.0,            # rolling average performance
    "total_rewards":0.0,
    "retrain_flag": False         # trigger retraining
}

In [ ]:
from langchain_core.runnables import RunnableConfig
config = RunnableConfig(recursion_limit=100000)

In [ ]:
graph = build_continuous_graph()
results_list=[]
# graph.invoke(inputs, config)
app = graph.compile()

for step_count in range(0,10):
  final_state = app.invoke(state,config)
  state=final_state
  results_list.append({
            "Step": step_count + 1,
            "Truck":  final_state['action']["truck"],
            "Route":  final_state['action']["route"],
            "Reward": final_state['reward'],
            "Avg_Reward": final_state['avg_reward'],
            "Explanation": final_state['decision_explained']
        })

🤖 RL chose → Truck: TRUCK_124, Route: R_013
📊 Feedback collected — reward: -10.00, avg: -2.00
✅ Performance acceptable, no retraining needed.


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

The RL model chose Truck TRUCK_261 for Route R_448. Explain the likely reasoning.
For this question, it is assumed that the customer is picking up a rental car from a station in downtown Seattle.
Step 1 - If no other destination is provided by the customer, then the trip must end with the destination being downtown Seattle.
🤖 RL chose → Truck: TRUCK_124, Route: R_013
📊 Feedback collected — reward: -10.00, avg: -5.90
✅ Performance acceptable, no retraining needed.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The RL model chose Truck TRUCK_124 for Route R_013. Explain the likely reasoning.
1 Answer | Add Yours
I agree with Mr. Hargreaves. In looking at the data you would expect that the most likely choice would be TRUCK_124, as you want to maximize the probability that the trucks arrive on
🤖 RL chose → Truck: TRUCK_488, Route: R_146
📊 Feedback collected — reward: -10.00, avg: -6.72
✅ Performance acceptable, no retraining needed.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The RL model chose Truck TRUCK_488 for Route R_146. Explain the likely reasoning.
I didn’t look at any of the RL model output in depth. I only looked at it to see the main results. But I’m surprised the model has not used the “main road” as the route,


In [ ]:
final_results=pd.DataFrame(results_list)

In [ ]:
final_results.to_csv("results.csv")